# Der Höhepunkt: SQL, Spark-Tables, Reporting ...

* SQL Abfragen auf Dataframes anwenden
* Mit Spark Datenbanken & Tabellen arbeiten
* User-Defined Functions benutzen
* Datenmegen joinen
* Window Operationen
* Testdaten erzeugen, welche zwei Geschäftsprozesse aus der Versicherungswelt abbilden
* Die ersten (und wesentlichsten) Schritte für ein sinnvolles Monitoring dieser Prozesse gehen


## Wie immer eine Spark-Application registieren

In [1]:
import findspark
findspark.init()
findspark.find()

'/usr/local/lib/python3.10/dist-packages/pyspark'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = (
    SparkSession
        .builder
        .appName("sql")
        .config("spark.dynamicAllocation.enabled",False)
        .config("spark.sql.adaptive.enabled",False)
        .master("local[4]")
        .getOrCreate()
)
spark

23/09/04 11:07:01 WARN Utils: Your hostname, pupil-a resolves to a loopback address: 127.0.1.1; using 23.88.105.62 instead (on interface eth0)
23/09/04 11:07:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/04 11:07:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Nun Testdaten erzeugen
Hierzu haben wir ein Beispiel aus der Versicherungswelt vorbereiten. Vielleicht ist es dem in der Signal sogar ein bisschen ähnlich ;-)


![](architecture-level-01.dio.svg)

In [3]:
import generate_test_data
generate_test_data.main(1000) # führe das nur _einmal_ aus!

Wir sehen nun auf im Dateibrowser eine Liste von Dateien, die überwiegend fachliche Events enthalten, die zwei Geschäftsprozesse abbilden:

### Ein Kunden bekommt einen neuen Vertrag

Die Systeme welche die Events schreiben liegen in der Verantwortung verschiedener Teams:

#### Team Antrag
 * **Antrag Erzeugt:** Der Kunde geht im Internet auf ein System um einen Antrag auf einen Versicherungsvertrag zu stellen. Dieses System validert schon ein wenig, versucht einen schon bestehenden Kunden im Partner-Systeme zu finden und schmeißt im Anschluss dieses Business-Event.
 * **AB-Test:** das Team Antrag ist stehst bemüht die User-Experience zu optimieren. So will es zum Beispiel die Durchlaufzeit auf ihrem eigenen System zu verkürzen. So variert es z.B. die Farbe des Knopfes. Wenn es bei einem Antrag von der Standardfarbe des Knopfes abweicht, schreibt es dieses technische Event
 * **Kunde hat Angebot aktzeptiert:** Nachdem das Vertragssystem sein grünes Licht gegeben hat, wurde der Vertrag vom Kunden auch akzeptiert.
 * **Kunde hat Angebot abgelehnt:** Wie vorhergehendes Event nur der Kunde will das Angebot nicht annehmen.
 
#### Team Vertrag
 * **Antrag abgelehnt**: Ein Vertrag kann vom Vertragssystem abgelehnt werden. Dieses fragt z.B. noch bei Fraud an und validert auch sonst den Antrag viel "besser" als es das Antragssystem könnte.
 * **Kunde angelegt**: (eigentlich müsste ein anderes System dieses Event schreiben, aber in unserem Beispiel soll das mal OK sein) Wenn in einem Antrag nicht schon eine Referenz auf einen Kunden vorhanden ist, dann legt das Vertragssystem einen Kunden an und informiert Gott und die Welt hierüber.
 * **Vertrag angeboten**: Wenn ein Vertragsantrag aus Sicht des Vertragssystem gut aussieht, dann gibt es den Vertrag frei zur Annahme durch den Kunden. 
 * **Vertrag policiert**: Wenn der Kunden ein Vertragsangebot akzeptiert hat, policieren wir den Vertrag. Nun ist der Kunde versichert. Jetzt können bedenkenlos Schadensfälle eintreten.
 
#### Team Schaden
 * **Schaden reguliert**: wenn ein Schadensfall geprüft wurde und alles passt, dann überweisen wir Geld

#### Übrigens
**kunden.csv** ist nur ein Container mit allen Kunden ;-)



## Erste Blicke in unsere Testdaten

In [5]:
kunden_df = spark.read.option("header", True).option("inferSchema", True).csv("kunden.csv")
kunden_df.printSchema()
kunden_df.show(4, truncate=False)

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)

+------------------------------------+-----------------+
|id                                  |name             |
+------------------------------------+-----------------+
|ce4cf720-4a1a-11ee-8349-7b31dfcc7252|gorzala          |
|d55a2ede-4a1a-11ee-8cce-b739d8c6e256|Scharrenberch    |
|793401ed-7462-4201-9846-eaf540936914|Gracious Jackson |
|ecc213f5-96cd-42c4-a018-d60d98064803|Mystifying Edison|
+------------------------------------+-----------------+
only showing top 4 rows



Soweit so bekannt. Wir könnten jetzt auf die Daten zugreifen in dem wir wie in den vorhergegangen Kapiteln filter anwenden, Null-Werte entfernen usw.
Wir möchten aber absofort ganz normal mit SQL auf unsere Daten zugreifen. Dazu müssen wir zu einem Dataframe in Spark einen _View_ anlegen. Diese Views werden dann ganz normal wie Datenbanktabellen in SQL-Datenbanken behandelt (read-only).

In [6]:
kunden_df.createOrReplaceTempView("kunden")

**Beachte:** die Funtion gibt nichts zurück. In der Spark-Application ist jetzt eine eine Tabelle unter dem Namen "kunden" verfügbar!

In [7]:
spark.sql("SELECT * FROM kunden")

DataFrame[id: string, name: string]

Wir sehen das wir ein Dataframe zurückbekommen. Auf diesem können wir dann wieder die bekannten Operationen ausführen. Beachte, ob Du die Dateframe-API oder die SQL-API nutzt, hat in der Regel **keinen** Impact auf die Performance!

**Übung** setzte hier mal ein paar SQLs gegen diese Tabelle ab. Filter nach bestimmten Namen... 

In [8]:
spark.sql('SELECT * FROM kunden').show()

+--------------------+-----------------+
|                  id|             name|
+--------------------+-----------------+
|ce4cf720-4a1a-11e...|          gorzala|
|d55a2ede-4a1a-11e...|    Scharrenberch|
|793401ed-7462-420...| Gracious Jackson|
|ecc213f5-96cd-42c...|Mystifying Edison|
|abbcb555-a501-433...| Goofy Hofstadter|
|0cef6d61-b4a2-48e...|  Relaxed Meitner|
|a5b96d1f-8f2a-487...|    Gallant Moore|
|18010b35-daff-42b...|   Tender Noether|
|8e6324df-094a-44c...|    Tender Galois|
|1d247f77-251e-425...|    Crazy Poitras|
|bb8b1ec7-a7b4-49b...|Magical Ramanujan|
|e9fbd5b7-8da2-4aa...|     Frosty Tharp|
|09aa0bae-1e51-40a...|  Festive Hellman|
|016e722d-64a6-494...|        Epic Cori|
|8555b696-5cda-4f3...|    Great Hellman|
|e2fb710f-2036-497...|Xenodochial Gould|
|784161e0-9e1a-40c...|Suspicious Turing|
|6ccab604-aa98-421...|    Happy Hellman|
|07bb7d22-ebf6-476...|    Reverent Pike|
|e11ee24e-c219-4d1...|    Infallible Wu|
+--------------------+-----------------+
only showing top

## Alle Testdaten importieren
und views anlegen

In [9]:
# AB-test
ab_test_schema = "AntragsId STRING"
ab_test_df = spark.read.option("header", True).schema(ab_test_schema).csv("ab_test.csv")
ab_test_df.createOrReplaceTempView("ab_test")

# Antrag Abgelehnt
antrag_abgelehnt_schema = "AntragsId STRING, KundenId STRING, TimeStamp TIMESTAMP, Grund STRING"
antrag_abgelehnt_df = spark.read.option("header", True).schema(antrag_abgelehnt_schema).csv("antrag_abgelehnt.csv")
antrag_abgelehnt_df.createOrReplaceTempView("antrag_abgelehnt")

# Antrag Erzeugt
antrag_erzeugt_schema = "AntragsId STRING, StartZeit TIMESTAMP, EndZeit TIMESTAMP, KundenId STRING"
antrag_erzeugt_df = spark.read.option("header", True).schema(antrag_erzeugt_schema).csv("antrag_erzeugt.csv")
antrag_erzeugt_df.createOrReplaceTempView("antrag_erzeugt")

# Kunde Angelegt
kunde_angelegt_schema = "KundenId STRING, TimeStamp TIMESTAMP"
kunde_angelegt_df = spark.read.option("header", True).schema(kunde_angelegt_schema).csv("kunde_angelegt.csv")
kunde_angelegt_df.createOrReplaceTempView("kunde_angelegt")

# Kunde hat Angebot Abgelehnt
kunde_hat_angebot_abgelehnt_schema = "VertragsId STRING, AntragsId STRING, Grund STRING, TimeStamp TIMESTAMP"
kunde_hat_angebot_abgelehnt_df = spark.read.option("header", True).schema(kunde_hat_angebot_abgelehnt_schema).csv("kunde_hat_angebot_abgelehnt.csv")
kunde_hat_angebot_abgelehnt_df .createOrReplaceTempView("kunde_hat_angebot_abgelehnt")

# Kunde hat Angebot Akzeptiert
kunde_hat_angebot_akzeptiert_schema = "VertragsId STRING, AntragsId STRING, KundenId STRING, TimeStamp TIMESTAMP"
kunde_hat_angebot_akzeptiert_df = spark.read.option("header", True).schema(kunde_hat_angebot_akzeptiert_schema).csv("kunde_hat_angebot_akzeptiert.csv")
kunde_hat_angebot_akzeptiert_df.createOrReplaceTempView("kunde_hat_angebot_akzeptiert")

# Schaden Gemeldet
schaden_gemeldet_schema = "VertagsId STRING, SchadensId STRING, Schadenshoehe INT, TimeStamp TIMESTAMP"
schaden_gemeldet_df = spark.read.option("header", True).schema(schaden_gemeldet_schema).csv("schaden_gemeldet.csv")
schaden_gemeldet_df.createOrReplaceTempView("schaden_gemeldet")

# Schaden Reguliert
schaden_reguliert_schema = "SchadensId STRING, TimeStamp TIMESTAMP"
schaden_reguliert_df = spark.read.option("header", True).schema(schaden_reguliert_schema).csv("schaden_reguliert.csv")
schaden_reguliert_df.createOrReplaceTempView("schaden_reguliert")

# Vertrag Angeboten
vertrag_angeboten_schema = "VertragsId STRING, AntragsId STRING, KundenId STRING, TimeStamp TIMESTAMP"
vertrag_angeboten_df = spark.read.option("header", True).schema(vertrag_angeboten_schema).csv("vertrag_angeboten.csv")
vertrag_angeboten_df.createOrReplaceTempView("vertrag_angeboten")

# Vertrag Policiert
vertrag_policiert_schema = "VertragsId STRING, TimeStamp TIMESTAMP"
vertrag_policiert_df = spark.read.option("header", True).schema(vertrag_policiert_schema).csv("vertrag_policiert.csv")
vertrag_policiert_df.createOrReplaceTempView("vertrag_policiert")

In [10]:
# alles da?
spark.sql("SHOW TABLES").show(truncate=False)

+---------+----------------------------+-----------+
|namespace|tableName                   |isTemporary|
+---------+----------------------------+-----------+
|         |ab_test                     |true       |
|         |antrag_abgelehnt            |true       |
|         |antrag_erzeugt              |true       |
|         |kunde_angelegt              |true       |
|         |kunde_hat_angebot_abgelehnt |true       |
|         |kunde_hat_angebot_akzeptiert|true       |
|         |kunden                      |true       |
|         |schaden_gemeldet            |true       |
|         |schaden_reguliert           |true       |
|         |vertrag_angeboten           |true       |
|         |vertrag_policiert           |true       |
+---------+----------------------------+-----------+



In [11]:
# Stichprobe bzgl. der Datentypen?
spark.sql("DESCRIBE TABLE EXTENDED schaden_gemeldet").show()

+-------------+---------+-------+
|     col_name|data_type|comment|
+-------------+---------+-------+
|    VertagsId|   string|   null|
|   SchadensId|   string|   null|
|Schadenshoehe|      int|   null|
|    TimeStamp|timestamp|   null|
+-------------+---------+-------+



## Ein erstes Reporting

In [12]:
# Zeige alle Anträge an, bei den wir die Farbe des Knopfs verändert haben

# erstmal alle Anträge

spark.sql("SELECT count(1) FROM antrag_erzeugt").show()
spark.sql("""
     SELECT count(1)
       FROM antrag_erzeugt
 INNER JOIN ab_test
         ON antrag_erzeugt.AntragsId = ab_test.AntragsId
""").show()

+--------+
|count(1)|
+--------+
|    1000|
+--------+

+--------+
|count(1)|
+--------+
|     105|
+--------+



## Spark Tables

Bisher haben wir gesehen wie wir CSV Dateien laden und speichern können.
Dabei mussten wir allerdings z.B. immer ein Schema angeben.
Das ist auf Dauer lästig und mindestens aufwendig.


![](spark-catalog.dio.svg)

In [13]:
# beispiel mit dem Kunde angelegt Event
kunde_angelegt_df.printSchema()

root
 |-- KundenId: string (nullable = true)
 |-- TimeStamp: timestamp (nullable = true)



In [14]:
kunde_angelegt_df.write.mode("overwrite").saveAsTable("myTestTable")

In [15]:
kunde_angelegt_df = spark.read.table("myTestTable")

In [16]:
kunde_angelegt_df.printSchema()

root
 |-- KundenId: string (nullable = true)
 |-- TimeStamp: timestamp (nullable = true)



wir können aber auch direkt mit SQL aus einer Table lesen

In [17]:
spark.sql("SELECT * FROM myTestTable")

DataFrame[KundenId: string, TimeStamp: timestamp]

### Typen von Catalogs


* **In-memory Catalog** Wenn eine Spark-Sitzung ended, wird auch der Catalog aufgeräumt
* **Persistent Catalog** Metadaten werden permanent gespeichert, Spark hat einen [Hive](https://hive.apache.org/)-basierten Catalog eingebaut.

### Typen von Tables

* **Managed Tables** Spark kümmert sich um den Lebenszyklus. Es werden Daten und Schemata verwaltet. Nützlich für staging Daten. Wenn eine Tabelle gelöscht wird, dann werden Daten und Schemata gelöscht.
* **Unmanged Tables/External Tables** Das Schema wird auch hier von Spark gemanaged, aber die Daten in einer frei wählbaren location. Löschen einer Tabelle löscht hier nur das Schema. Nützlich für das Ergebniss von ETL Pipelines

In [18]:
import findspark
findspark.init()

from IPython.display import *
display(HTML("<style>pre { white-space: pre !important; }</style>"))

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = (
    SparkSession
        .builder
        .appName("SparkTablesApp")
        .master("local[4]")
        .config("spark.dynamicAllocation.enabled", "false")
        .config("spark.sql.adaptive.enabled", "false")
        
        .enableHiveSupport() # <---------------------------------
    
        .getOrCreate()
)

sc = spark.sparkContext
spark

23/09/04 12:08:13 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### Nun eine Datenbank in Hive erstellen

In [19]:
spark.sql("""
  SHOW DATABASES
  """).show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [20]:
spark.sql("""
    CREATE DATABASE IF NOT EXISTS Foo
""").show()

++
||
++
++



In [21]:
spark.sql("""
    SHOW DATABASES
""").show()

+---------+
|namespace|
+---------+
|  default|
|      foo|
+---------+



In [22]:
kunde_angelegt_df.write.mode("overwrite").saveAsTable("foo.bar")
# wurde als managed table abgespeichert, da keine externe location angegeben wurde

In [23]:
spark.sql("""
    SHOW TABLES in foo
""").show(truncate=False)

+---------+----------------------------+-----------+
|namespace|tableName                   |isTemporary|
+---------+----------------------------+-----------+
|foo      |bar                         |false      |
|         |ab_test                     |true       |
|         |antrag_abgelehnt            |true       |
|         |antrag_erzeugt              |true       |
|         |kunde_angelegt              |true       |
|         |kunde_hat_angebot_abgelehnt |true       |
|         |kunde_hat_angebot_akzeptiert|true       |
|         |kunden                      |true       |
|         |schaden_gemeldet            |true       |
|         |schaden_reguliert           |true       |
|         |vertrag_angeboten           |true       |
|         |vertrag_policiert           |true       |
+---------+----------------------------+-----------+



In [24]:
spark.sql("""
SELECT * from foo.bar
""").show()

+--------------------+-------------------+
|            KundenId|          TimeStamp|
+--------------------+-------------------+
|ecc213f5-96cd-42c...|2023-09-10 12:35:52|
|0cef6d61-b4a2-48e...|2023-09-07 09:11:09|
|784161e0-9e1a-40c...|2023-09-11 07:18:43|
|07bb7d22-ebf6-476...|2023-09-06 11:38:30|
|e11ee24e-c219-4d1...|2023-09-05 09:30:03|
|50a9152a-5ee0-41c...|2023-09-07 21:43:58|
|0fbb6c1e-89a1-402...|2023-09-09 11:11:13|
|f7029b08-1af0-44f...|2023-09-06 10:26:25|
|cdf5f5db-6fd0-4ed...|2023-09-20 09:39:13|
|d4d3cb7d-258f-401...|2023-09-17 04:33:27|
|70efc788-9d7b-4df...|2023-09-09 09:23:01|
|07a8f7bd-6f25-4c4...|2023-09-10 06:29:38|
|e63e36a2-fdb7-4bd...|2023-09-17 11:56:09|
|33e5a05c-680e-4aa...|2023-09-03 05:43:06|
|afdc1487-286f-4be...|2023-09-06 09:21:01|
|3667b3ed-4339-426...|2023-09-11 13:25:35|
|aa5ea83d-7860-467...|2023-09-07 04:29:21|
|341152a4-e731-4b8...|2023-09-06 23:18:30|
|e998f425-8f52-4ef...|2023-09-13 08:31:26|
|5a96afb2-4f0d-4c3...|2023-09-06 00:28:19|
+----------

In [25]:
# oder auch die tablle direkt in ein Dataframe lesen

tmp = spark.read.table("foo.bar")
tmp.printSchema()
tmp.show(2)

root
 |-- KundenId: string (nullable = true)
 |-- TimeStamp: timestamp (nullable = true)

+--------------------+-------------------+
|            KundenId|          TimeStamp|
+--------------------+-------------------+
|ecc213f5-96cd-42c...|2023-09-10 12:35:52|
|0cef6d61-b4a2-48e...|2023-09-07 09:11:09|
+--------------------+-------------------+
only showing top 2 rows



In [26]:
# Nun schauen wir uns mal die details der Tabelle an
spark.sql("""
    DESCRIBE TABLE EXTENDED foo.bar
""").show(truncate=False)

+----------------------------+----------------------------------------------------------------------+-------+
|col_name                    |data_type                                                             |comment|
+----------------------------+----------------------------------------------------------------------+-------+
|KundenId                    |string                                                                |null   |
|TimeStamp                   |timestamp                                                             |null   |
|                            |                                                                      |       |
|# Detailed Table Information|                                                                      |       |
|Catalog                     |spark_catalog                                                         |       |
|Database                    |foo                                                                   |       |
|Table    

In [27]:
# Nun eine UNMANAGED Table anlegen
(
    kunden_df
        .write
        .mode("overwrite")
        .option("path", "/home/pupil/spark-course/course/03-SQL/spark-warehouse/persistent/kunden.parquet")
        #.option("format", "csv") # defaults to parquet
        .saveAsTable("foo.bar")
)

In [28]:
spark.sql("""
 DESCRIBE TABLE EXTENDED foo.bar
""").show(truncate=False)

+----------------------------+---------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                              |comment|
+----------------------------+---------------------------------------------------------------------------------------+-------+
|id                          |string                                                                                 |null   |
|name                        |string                                                                                 |null   |
|                            |                                                                                       |       |
|# Detailed Table Information|                                                                                       |       |
|Catalog                     |spark_catalog                                                                    

In [29]:
spark.sql("""
DROP TABLE foo.bar
""").show(truncate=False)

++
||
++
++



im datei browser anschauen, dass die Datein nicht wirklich gelöscht wurden
Nun mit SQL die Tabelle wieder herstllen


In [30]:
# Nun eine Table direkt aus parquet herstellen (parquet hat Schema eingebaut ;-) )
spark.sql("""
    CREATE TABLE  foo.bar
    USING PARQUET
    LOCATION "/home/pupil/spark-course/course/03-SQL/spark-warehouse/persistent/kunden.parquet"
""").show(truncate=False)

++
||
++
++



In [31]:
spark.sql("""
    SELECT * FROM foo.bar
""").show()

+--------------------+-----------------+
|                  id|             name|
+--------------------+-----------------+
|ce4cf720-4a1a-11e...|          gorzala|
|d55a2ede-4a1a-11e...|    Scharrenberch|
|793401ed-7462-420...| Gracious Jackson|
|ecc213f5-96cd-42c...|Mystifying Edison|
|abbcb555-a501-433...| Goofy Hofstadter|
|0cef6d61-b4a2-48e...|  Relaxed Meitner|
|a5b96d1f-8f2a-487...|    Gallant Moore|
|18010b35-daff-42b...|   Tender Noether|
|8e6324df-094a-44c...|    Tender Galois|
|1d247f77-251e-425...|    Crazy Poitras|
|bb8b1ec7-a7b4-49b...|Magical Ramanujan|
|e9fbd5b7-8da2-4aa...|     Frosty Tharp|
|09aa0bae-1e51-40a...|  Festive Hellman|
|016e722d-64a6-494...|        Epic Cori|
|8555b696-5cda-4f3...|    Great Hellman|
|e2fb710f-2036-497...|Xenodochial Gould|
|784161e0-9e1a-40c...|Suspicious Turing|
|6ccab604-aa98-421...|    Happy Hellman|
|07bb7d22-ebf6-476...|    Reverent Pike|
|e11ee24e-c219-4d1...|    Infallible Wu|
+--------------------+-----------------+
only showing top

**Take Away** Spark Tables machen die Entwicklung schneller und einfacher

## Spark User Defined Functions

Spark stellt dir zwar eine Menge an Funktionen zur Datenmanipulation zur Verfügung aber machnchmal fehlt doch etwas.
Aus diesem Grund gibt es User Defined Funktions.

In [32]:
spark.sql("""
  SELECT * FROM antrag_erzeugt
""").show(truncate=False)


+------------------------------------+-------------------+-------------------+------------------------------------+
|AntragsId                           |StartZeit          |EndZeit            |KundenId                            |
+------------------------------------+-------------------+-------------------+------------------------------------+
|4713b4b8-e077-4c7a-8e7d-7306786c9773|2023-09-06 20:48:00|2023-09-06 20:57:00|793401ed-7462-4201-9846-eaf540936914|
|e0337bbc-8bc7-4ff8-8e86-e495c40dd469|2023-09-10 12:32:52|2023-09-10 12:34:52|null                                |
|ee2dd754-84f2-4f5c-89da-26e1a5c6f41b|2023-09-03 17:11:44|2023-09-03 17:19:44|null                                |
|bb3c489a-8358-4aca-b6cd-c016a40baa9b|2023-09-05 18:00:55|2023-09-05 18:07:55|abbcb555-a501-4333-a73e-4f147a61b65b|
|657c4d3d-e2f6-4705-8f0a-53e4eb23b96e|2023-09-07 09:01:09|2023-09-07 09:10:09|null                                |
|8620d538-b37f-4835-ad3c-8f887cb36ebd|2023-09-03 01:02:45|2023-09-03 01:

Wir möchten hier zum Beispiel die Zeitauer als eigene Spalte haben, die der Kunde auf der Antragsstrecke verbracht hat. Das würde zwar auch mit SQL gehen, aber soll hier als Beispiel dienen.

**Beachte**: Für Spark sind UDFs Blackboxen und Spark wendet deswegen keinerlei Optimierungen auf deren Anwendung an.

In [63]:
# meine erste UDF
def duration(first_ts, second_ts):
    delta = second_ts - first_ts
    return int(delta.total_seconds())

In [70]:
duration_udf = udf( lambda first, second: duration(first,second), IntegerType()) 

### Anwenden in DF-Code

In [69]:
antrag_erzeugt_df.select(
    "*",
    duration_udf(col("StartZeit"), col("EndZeit")).alias("Duration")
).show(5, truncate=False)

+------------------------------------+-------------------+-------------------+------------------------------------+--------+
|AntragsId                           |StartZeit          |EndZeit            |KundenId                            |Duration|
+------------------------------------+-------------------+-------------------+------------------------------------+--------+
|4713b4b8-e077-4c7a-8e7d-7306786c9773|2023-09-06 20:48:00|2023-09-06 20:57:00|793401ed-7462-4201-9846-eaf540936914|540     |
|e0337bbc-8bc7-4ff8-8e86-e495c40dd469|2023-09-10 12:32:52|2023-09-10 12:34:52|null                                |120     |
|ee2dd754-84f2-4f5c-89da-26e1a5c6f41b|2023-09-03 17:11:44|2023-09-03 17:19:44|null                                |480     |
|bb3c489a-8358-4aca-b6cd-c016a40baa9b|2023-09-05 18:00:55|2023-09-05 18:07:55|abbcb555-a501-4333-a73e-4f147a61b65b|420     |
|657c4d3d-e2f6-4705-8f0a-53e4eb23b96e|2023-09-07 09:01:09|2023-09-07 09:10:09|null                                |540     |


### Anwenden in SQL

In [73]:
# für spark sql registrieren
spark.udf.register("duration", duration, IntegerType())

23/09/04 12:31:45 WARN SimpleFunctionRegistry: The function duration replaced a previously registered function.


<function __main__.duration(first_ts, second_ts)>

In [75]:
spark.sql("""
    SELECT 
        *,
        duration(StartZeit, Endzeit) AS Foo
    FROM antrag_erzeugt
""").show(4, truncate=False)

+------------------------------------+-------------------+-------------------+------------------------------------+---+
|AntragsId                           |StartZeit          |EndZeit            |KundenId                            |Foo|
+------------------------------------+-------------------+-------------------+------------------------------------+---+
|4713b4b8-e077-4c7a-8e7d-7306786c9773|2023-09-06 20:48:00|2023-09-06 20:57:00|793401ed-7462-4201-9846-eaf540936914|540|
|e0337bbc-8bc7-4ff8-8e86-e495c40dd469|2023-09-10 12:32:52|2023-09-10 12:34:52|null                                |120|
|ee2dd754-84f2-4f5c-89da-26e1a5c6f41b|2023-09-03 17:11:44|2023-09-03 17:19:44|null                                |480|
|bb3c489a-8358-4aca-b6cd-c016a40baa9b|2023-09-05 18:00:55|2023-09-05 18:07:55|abbcb555-a501-4333-a73e-4f147a61b65b|420|
+------------------------------------+-------------------+-------------------+------------------------------------+---+
only showing top 4 rows



### Übung, schreibe eine UDF, die die Dauer Cluster

* alles unter 200 Sekunden "grün"
* zwischen 200 und 500 "gelb"
* der rest rot

## Joins

## Windows